# Cot (chain of Though)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai
client = openai.OpenAI()

### 일반 프롬프트 vs Zero-shot Cot

In [3]:
# 일반 프롬프트
prompt = '123 X 31은 얼마인가요?'

# Zero-shot Cot 적용 프롬프트
cot_prompt = '단계별로 계산해주세요: 123 X 31은 얼마인가요?'

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role":"user", "content": prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role":"user", "content": cot_prompt}]
)

In [6]:
res.choices[0].message.content

'123 X 31 = 3813'

In [9]:
print(cot_res.choices[0].message.content)

123 X 31의 연산을 단계별로 보자면, 

1단계: 1 * 123 = 123
2단계: 3 * 123 = 369 (하지만 3은 십의 자리이므로 369에 10을 곱해줍니다. 그래서 3690이 됩니다.)
3단계: 123 (1단계 값) + 3690 (2단계값) = 3813

따라서, 123 X 31 = 3813입니다.


# Few-shot Cot

In [12]:
few_shot_cot = """ 
Q: 123 x 31 은 얼마인가요?
A: 
1. 123를 분해합니다: 123 = 100 + 20 + 3
2. 각 항을 31과 곱합니다:
   - 100 x 31 = 3100
   - 20 x 31 = 620
   - 3 x 31 = 93

3. 이제 이 결과들을 더합니다:
   - 3100 + 620 + 93

4. 계산을 진행하면:
   - 3100 + 620 = 3720
   - 3720 + 93 = 3813 

따라서, 123 x 31 = 3813입니다.

Q: 789 X 56은 얼마 인가여?
A:
"""

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": few_shot_cot}]
)

In [13]:
print(res.choices[0].message.content)

1. 789를 분해합니다: 789 = 700 + 80 + 9
2. 각 항을 56과 곱합니다:
   - 700 x 56 = 39200
   - 80 x 56 = 4480
   - 9 x 56 = 504

3. 이제 이 결과들을 더합니다:
   - 39200 + 4480 + 504 

4. 계산을 진행하면:
   - 39200 + 4480 = 43680
   - 43680 + 504 = 44184 

따라서, 789 x 56 = 44184입니다.


### 논리 문제 해결

In [14]:
prompt = "앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다. 프랭크(Frank)는 데이비드(David)보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"

cot_prompt = "단계적으로 생각해서 대답해주세요: 앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다. 프랭크(Frank)는 데이비드(David)보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"

In [15]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role":"user", "content": prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role":"user", "content": cot_prompt}]
)

In [16]:
print(res.choices[0].message.content)

가장 어린 사람은 프랭크(Frank)입니다.


In [17]:
print(cot_res.choices[0].message.content)

가장 어린 사람을 찾기 위해, 우선 다음과 같은 관계를 정의해보겠습니다:

1. 앨리스는 밥보다 나이가 많다.
2. 밥은 찰리보다 나이가 많다.
3. 데이비드는 앨리스보다 어리고, 에밀리보다 많다.
4. 찰리는 프랭크보다 많다.
5. 에밀리는 찰리보다 많고, 밥보다 어리다.
6. 프랭크는 데이비드보다 어리다.

이를 기반으로 다음과 같이 나이의 순서를 파악할 수 있습니다:

앨리스 > 밥 > 찰리 > 프랭크

에밀리 > 찰리 > 프랭크

데이비드 > 에밀리

데이비드 > 프랭크

따라서 가장 어린 사람은 프랭크(Frank)입니다.


### 코드 디버깅

In [18]:
prompt = """ 
다음과 같이 코드를 작성했더니 에러가 발생합니다. 왜 일까요?

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

cot_prompt = """ 
다음과 같이 코드를 작성했더니 에러가 발생합니다. 
실수가 무억인지 찾아 단계별로 설명해주세요

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

In [19]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role":"user", "content": prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role":"user", "content": cot_prompt}]
)

In [20]:
print(res.choices[0].message.content)

위의 코드에서 파이썬은 들여쓰기를 강조하기 때문에 에러가 발생합니다. "return a + b"가 함수 "add_numbers(a, b)"의 본문(body)을 형성하므로, 이는 함수 선언 다음줄에서 추가적인 들여쓰기(indentation)가 필요합니다.

아래는 제대로 작동하는 코드입니다:

```python
def add_numbers(a, b):
    return a + b

print(add_numbers(1, 2))
```
이렇게 작성하면 정상적으로 동작하며, 출력결과는 3이 나올 것입니다.


In [21]:
print(cot_res.choices[0].message.content)

이 코드에서는 주로 들여쓰기에 대한 에러(SyntaxError: expected an indented block)가 발생합니다.

파이썬은 블록(block)을 구분할 때 중괄호나 키워드 대신 들여쓰기를 사용합니다.
따라서, 함수의 본문은 들여쓰기를 해줘야 합니다.

아래와 같이 코드를 수정하면 에러 없이 정상적으로 실행됩니다.

```python
def add_numbers(a, b):
    return a + b  # <-- 이 줄이 들여써져야 함

print(add_numbers(1, 2))
```

이렇게 들여쓰기를 한 후에는 결과값으로 3을 반환받아 올 수 있습니다.
